In [ ]:
%run "0.0 Data preparation.ipynb"

# Dataset selection

In [ ]:
_dataName, _inputData, _dataNameSUSNormalized, _inputDataSUSNormalized = selectDataset("data20190703")

In [ ]:
stackedBarPlotsFilenamePathStem = graphsSavePathStem + "/stacked-bar-plots"
tryCreateFolder(stackedBarPlotsFilenamePathStem)

barPlotsErrorBarsFilenamePathStem = graphsSavePathStem + "/bar-plots-with-error-bars"
tryCreateFolder(barPlotsErrorBarsFilenamePathStem)

In [ ]:
def plotStackedBar(question
                   , dataName=_dataName
                   , saveFig=False
                   , filename=None
                   , fig=None
                   , ax=None
                   , title=None
                   , displayedGameNames=games
                   , showLegend=True
                  ):

    assert((dataName in datasets), "Not found in datasets: '" + dataName + "'")

    data = datasets[dataName]

    callShow = False

    if None == ax:
        callShow = True
        if None == fig:
            fig = plt.figure()
        ax = plt.subplot(111)

    #    assert(gameIndex < len(games), "game index must be smaller than " + str(len(games)))

    ind = np.arange(len(games))    # the x locations for the groups
    width = 0.35       # the width of the bars: can also be len(x) sequence
    colors = cm.jet(np.linspace(1, 0, 5))

    #games
    pg = [[] for i in range(len(games))]
    participantCounts = [0 for i in range(len(games))]

    for gameIndex in range(len(games)):

    #        print("gameIndex="+str(gameIndex))
        game = games[gameIndex]
        data = datasets[dataName]
        data = data.loc[:, [question, gameQuestion]].groupby([gameQuestion, question]).size()
        data = data[game]

        #pd.Series(("1", "2", "3", "4", "5")).apply(lambda x: data.get(x, 0))
        counts1 = data.get(1,0)
        counts2 = data.get(2,0)
        counts3 = data.get(3,0)
        counts4 = data.get(4,0)
        counts5 = data.get(5,0)
        participantCounts[gameIndex] = data.sum()

        #agreement scale: 0 == 100% agree, 4 == 100% disagree
        pg[gameIndex] = [[] for i in range(5)]

        pg[gameIndex][0] = ax.bar(\
                              ind[gameIndex],
                              counts1,\
                              width,\
                              color=colors[0],
                              bottom=counts5+counts4+counts3+counts2,\
                              )
        pg[gameIndex][1] = ax.bar(\
                              ind[gameIndex],
                              counts2,\
                              width,\
                              color=colors[1],
                              bottom=counts5+counts4+counts3,\
                              )
        pg[gameIndex][2] = ax.bar(\
                              ind[gameIndex],
                              counts3,\
                              width,\
                              color=colors[2],
                              bottom=counts5+counts4,\
                              )
        pg[gameIndex][3] = ax.bar(\
                              ind[gameIndex],
                              counts4,\
                              width,\
                              color=colors[3],
                              bottom=counts5,\
                              )
        pg[gameIndex][4] = ax.bar(\
                              ind[gameIndex],\
                              counts5,\
                              width,\
                              color=colors[4],\
                              )

    plt.ylabel('Stacked answers')
    if not title:
        plt.title(question)
    else:
        plt.title(title)

    # margins left and right of the bars from the Y axis
    #    plt.margins(1.2)

    plt.xticks(ind, displayedGameNames)
    #    plt.yticks(np.arange(0, data.sum(), round(max(participantCounts)/10)))
    if showLegend:
        plt.legend((pg[gameIndex][0][0], pg[gameIndex][1][0], pg[gameIndex][2][0], pg[gameIndex][3][0], pg[gameIndex][4][0])
            , (1,2,3,4,5),
            loc='center left', bbox_to_anchor=(1, 0.5))

    plt.tight_layout()

    if callShow:
        plt.show()

    if saveFig:
        shortQuestion = shortQuestions.index[shortQuestions.values==question].values[0]
        if filename==None:
            path = stackedBarPlotsFilenamePathStem + "/" + dataName
            tryCreateFolder(path)
            filename = path + "/" + shortQuestion
        fig.savefig(filename)

plotStackedBar(indexedQuestions[4], dataName=_dataName, saveFig=False)

In [ ]:
fig = plt.figure(figsize=(15,8))
graphIndex = 1

for question in shortLikertQuestions:
    ax = fig.add_subplot(3,4,graphIndex)
    shortQuestion = shortQuestions.index[shortQuestions.values==question].values[0]
    plotStackedBar(question
                   , dataName=_dataName
                   , saveFig=False
                   , fig=fig
                   , ax=ax
                   , title=shortQuestion
                   , displayedGameNames=shortGameNames.values
                   , showLegend=(graphIndex==11)
                  )
    graphIndex += 1
path = stackedBarPlotsFilenamePathStem + "/" + _dataName
tryCreateFolder(path)
fig.savefig(path + "/matrixStackedBars")

In [ ]:
for question in shortLikertQuestions:
    plotStackedBar(question, dataName=_dataName, saveFig=False)

In [ ]:
# test to check values displayed in the bar plots
data = _inputData.loc[:, [indexedQuestions[4], gameQuestion]].groupby([gameQuestion, indexedQuestions[4]]).size()
data['Dr Bug: Microbe Mayhem']

### SUS Likert scale score variance

In [ ]:
saveFig = True

for question in indexedLikertQuestions:
    fig = plt.figure()
    ax = fig.add_subplot(111)
    sns.barplot(x=gameQuestion, y=question, data=_inputData, ax = ax)
    ax.set_xticklabels(shortGameNames)
    plt.xlabel(question)
    plt.ylabel('Agreement - Likert scale')
    plt.ylim(1, 5)

    if saveFig:
        shortQuestion = shortQuestions.index[shortQuestions.values==question].values[0]
        path = barPlotsErrorBarsFilenamePathStem + "/" + _dataName
        tryCreateFolder(path)
        fig.savefig(path + "/" + shortQuestion)

In [ ]:
saveFig = True

fig = plt.figure(figsize=(15,8))
graphIndex = 1

for question in indexedLikertQuestions:

    ax = fig.add_subplot(3,4,graphIndex)
    graphIndex += 1
    sns.barplot(x=gameQuestion, y=question, data=_inputData, ax = ax)
    shortQuestion = shortQuestions.index[shortQuestions.values==question].values[0]
    plt.xlabel('')
    plt.ylabel('')
    ax.set_xticklabels(['',shortQuestion,''])
    plt.ylim(1, 5)

if saveFig:
    path = barPlotsErrorBarsFilenamePathStem + "/" + _dataName
    tryCreateFolder(path)
    fig.savefig(path + "/SUS-11-bar-graphs-3x4-matrix")

In [ ]:
saveFig = True

fig = plt.figure(figsize=(15,8))
graphIndex = 1

for question in indexedLikertQuestions:

    ax = fig.add_subplot(3,4,graphIndex)
    graphIndex += 1
    sns.barplot(x=gameQuestion, y=question, data=_inputDataSUSNormalized, ax = ax)
    shortQuestion = shortQuestions.index[shortQuestions.values==question].values[0]
    plt.xlabel('')
    plt.ylabel('')
    ax.set_xticklabels(['',shortQuestion,''])
    plt.ylim(0, 4)

if saveFig:
    path = barPlotsErrorBarsFilenamePathStem + "/" + _dataName
    tryCreateFolder(path)
    fig.savefig(path + "/SUS-normalized-11-bar-graphs-3x4-matrix")

## Negative vs Positive questions comparison

Let's try and know whether the answers from negative and positive questions are indistinguishable.

In [ ]:
_inputDataSUSNormalized